In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
import tempfile
from IPython.display import Markdown, display, update_display

In [ ]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL_NAME = "gpt-4o"
openai = OpenAI()

In [ ]:
import os
import sys
import openai
import subprocess
import tempfile
import base64
import glob

# Assurez-vous d'avoir défini openai.api_key = "..." et MODEL_NAME = "...".
# Par exemple :
# openai.api_key = "sk-..."
# MODEL_NAME = "gpt-4"  # Ou "gpt-3.5-turbo", etc.

def encode_image(image_path):
    """
    Encode un fichier image en base64 (chaîne de caractères).
    """
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# --------------------------------------------------------------------
# 2) Fonctions pour générer le code à partir d'une preuve
# --------------------------------------------------------------------
def generate_test_code(proof_text):
    """
    Envoie la preuve mathématique à l'API OpenAI
    et récupère un code Python qui permet de tester ou de valider la preuve.
    """
    system_msg = (
        "Tu es un assistant IA spécialisé en mathématiques et en programmation. "
        "Tu vas recevoir une preuve mathématique, et tu dois générer du code Python "
        "pour la tester ou la valider expérimentalement. "
        "Le code doit inclure (au moins) la génération de données pertinentes, "
        "la logique de test ou de simulation, puis l'affichage ou l'export des résultats (texte/graphique)."
        "Inclus uniquement le code généré dans ta réponse, aucun commentaire en langage naturel. "
        "Assure-toi d'enregistrer toute image générée dans un dossier 'generated_outputs' pour que nous puissions la retrouver."
    )
    
    user_msg = (
        f"Voici la preuve mathématique proposée :\n\n{proof_text}\n\n"
        "Génère du code Python pour tester la validité de cette preuve, "
        "par simulation ou analyse. Le code doit produire un résumé "
        "des résultats, et, si possible, un graphique (enregistré dans le dossier 'generated_outputs' "
        "au format PNG)."
    )

    response = openai.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ],
        temperature=0.2  # Limiter la créativité pour un code plus "déterministe"
    )

    generated_code = response.choices[0].message.content
    return generated_code

# --------------------------------------------------------------------
# 3) Fonction pour exécuter le code généré
# --------------------------------------------------------------------
def run_generated_code(code):
    """
    Écrit le code dans un fichier temporaire et l'exécute dans un dossier
    où il pourra sauvegarder ses images. Capture stdout et stderr.
    """

    # On crée un répertoire "generated_outputs" si non existant
    output_dir = "generated_outputs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # On insère un petit snippet pour forcer le code à utiliser ce dossier
    # s'il fait un plt.savefig(...) par exemple. 
    # (Optionnel, si l'IA ne le fait pas déjà.)
    # On pourrait injecter du code, mais ici on se contente
    # de supposer que l'IA respectera le prompt.
    
    # Ecriture du code dans un fichier temporaire
    with tempfile.NamedTemporaryFile(suffix=".py", delete=False, mode='w', encoding='utf-8') as tmp_file:
        tmp_filename = tmp_file.name
        tmp_file.write(code)
    
    try:
        # Exécution du code dans le répertoire courant
        result = subprocess.run(
            ["python", tmp_filename],
            capture_output=True,
            text=True,
            check=False  # On met check=False pour capturer l'erreur sans lever l'exception
        )
        stdout = result.stdout
        stderr = result.stderr
    finally:
        os.remove(tmp_filename)
    
    return stdout, stderr

# --------------------------------------------------------------------
# 4) Fonction pour interpréter les résultats en streaming Markdown
#    + Possibilité de joindre une image (ou plusieurs) depuis generated_outputs
# --------------------------------------------------------------------
def interpret_results_streaming(proof_text, generated_code, stdout, stderr):
    """
    Envoie le code (generated_code) et les résultats de l'exécution (stdout, stderr) à l'API pour une interprétation textuelle
    au regard de la preuve fournie, en mode streaming.

    Cette fois, on va automatiquement scanner le dossier 'generated_outputs'
    pour chercher tous les .png. On les insère un par un dans le message.
    """
    system_msg = (
        "Tu es un assistant IA spécialisé en mathématiques et en interprétation de résultats de simulation. "
        "On te fournit la preuve initiale et le code d'une simulation ainsi que ses retours (generated_code, stdout, stderr). "
        "Donne une analyse de la cohérence entre la preuve, le code et les résultats, "
        "et retourne ta réponse au format Markdown."
    )

    # On construit un 'content' qui est un tableau 
    # (selon l'exemple de code que vous avez fourni).
    user_content = [
        {
            "type": "text",
            "text": (
                f"Preuve initiale :\n{proof_text}\n\n"
                f"Code de la simulation :\n{generated_code}\n\n"
                f"Résultats (stdout) :\n{stdout}\n\n"
                f"Erreurs éventuelles (stderr) :\n{stderr}\n\n"
                "Merci d'interpréter ces résultats et de conclure sur la preuve. "
                "Formule ta réponse de manière structurée en Markdown.\n"
            )
        }
    ]

    # On cherche toutes les images PNG qui auraient pu être générées 
    # dans le dossier "generated_outputs"
    output_dir = "generated_outputs"
    png_files = glob.glob(os.path.join(output_dir, "*.png"))

    # Pour chacune, on l'encode en base64 et on l'ajoute
    for png_path in png_files:
        encoded_img = encode_image(png_path)
        # On ajoute un bloc "image_url"
        user_content.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{encoded_img}"}
        })

    # Appel en mode streaming
    response_stream = openai.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_content}
        ],
        temperature=0.2,
        stream=True
    )
    
    # On débute un bloc Markdown
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in response_stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

# --------------------------------------------------------------------
# 5) Fonction principale (à appeler directement dans le notebook)
# --------------------------------------------------------------------
def main(proof_text: str):
    print("=== Génération du code Python pour tester la preuve... ===")
    test_code = generate_test_code(proof_text)

    # --- Nettoyage des backticks Markdown ---
    lines = test_code.splitlines()
    cleaned_lines = []
    for line in lines:
        if line.strip().startswith("```"):
            continue
        cleaned_lines.append(line)
    test_code = "\n".join(cleaned_lines).strip()
    # ----------------------------------------

    print("\n=== Code généré (nettoyé) : ===")
    print(test_code)

    print("\n=== Exécution du code généré... ===")
    stdout, stderr = run_generated_code(test_code)

    print("\n=== Sortie standard (stdout) : ===")
    print(stdout)
    if stderr.strip():
        print("\n=== Erreurs (stderr) : ===")
        print(stderr)

    print("\n=== Interprétation des résultats (streaming en Markdown) ===")
    interpret_results_streaming(proof_text, test_code, stdout, stderr)
    


In [ ]:
mon_texte_de_preuve = """
Ci-dessous, je propose une ébauche (relativement détaillée) d’une approche formelle pour construire un objet « cosmohedron-like » et démontrer (ou du moins argumenter rigoureusement) qu’il possède des propriétés fractales dans une limite bien définie. Attention : dans l’état actuel des recherches, la littérature ne fournit pas (à ma connaissance) de démonstration largement reconnue établissant qu’un « vrai » cosmohedron est strictement fractal. Ce que je vais donc présenter est un modèle mathématisé inspiré des idées de cosmohedra et de leur possible fractalité, en détaillant :
La définition axiomatique (ou construction) d’une famille {Cn}n∈N\{C_n\}_{n \in \mathbb{N}}{Cn​}n∈N​ d’objets géométriques (polytopes) qui tendent vers une limite.
Les propriétés de self-similarité ou d’auto-similarité approchée qui sont au cœur d’une structure fractale.
Une preuve (ou un argument) de non-invariance d’échelle integer-valued (c’est-à-dire que la dimension n’est pas un entier fixe) en s’appuyant sur une analyse de la « taille » de l’objet à différentes échelles.
Une conclusion sur la (quasi-)fractalité ou la fractalité effective de la limite de {Cn}\{C_n\}{Cn​}.

1. Définition d’une famille de polytopes « cosmohedron-like »
1.1. Construction combinatoire
Les cosmohedra (au sens de la littérature actuelle) se définissent via des données combinatoires et cinématiques (angles, énergie, moment, variables conformes, etc.) associées à la fonction d’onde cosmologique. Pour formaliser, on peut s’inspirer d’une définition axiomatique :
On part d’un polygone (ou polytope) de base Π0\Pi_0Π0​ en dimension ddd (avec d≥2d \ge 2d≥2, souvent la dimension 2 sert d’analogue).
On définit une liste de coupes ou « partitions » (généralisant la notion de triangulation) qui correspond, dans le contexte de la fonction d’onde cosmologique, aux factorisations possibles en sous-problèmes (un parallèle direct avec les associaèdres pour les amplitudes).
À chaque coupure ou partition, on associe des variables α,β,…\alpha, \beta,\dotsα,β,… (analogues à des Mandelstam variables ou à des invariants kinematiques).
Le cosmohedron en tant que polytope est l’intersection d’un certain nombre d’inégalités linéaires (et éventuellement quadratiques, selon les modèles) provenant de ces variables : { x∈RN:Mi(x)≥0 ∀i∈I}, \big\{\, x\in \mathbb{R}^N : M_i(x)\ge 0 \ \forall i \in I\big\},{x∈RN:Mi​(x)≥0 ∀i∈I}, où chaque MiM_iMi​ capture une condition « physique » ou « combinatoire » de consistance.
Pour exhiber la structure fractale, on définit une famille {Cn}\{C_n\}{Cn​} (analogues aux cosmohedra de plus en plus « raffinés ») via un processus itératif :
Initialisation : C1C_1C1​ est un polytope de dimension ddd (par exemple, un associaèdre standard ou un polygone en 2D).
Itération : Pour passer de CnC_nCn​ à Cn+1C_{n+1}Cn+1​, on effectue :
L’introduction de nouvelles variables cinématiques (exemple : scission supplémentaire de l’énergie dans un canal de Feynman),
L’ajout de nouvelles inégalités linéaires (ou combinatoires) qui forcent la convexité dans un espace plus grand,
Le tout aboutit à un polytope Cn+1⊂RNn+1C_{n+1}\subset \mathbb{R}^{N_{n+1}}Cn+1​⊂RNn+1​.
De cette manière, dim⁡(Cn)\dim(C_n)dim(Cn​) augmente (ou du moins, l’espace ambiant RNn\mathbb{R}^{N_n}RNn​ grandit), tandis que la projection de Cn+1C_{n+1}Cn+1​ sur certains sous-espaces ressemble de plus en plus à plusieurs copies (déformées) du polytope CnC_nCn​. C’est cette “auto-similarité” (même si souvent approchée et non exacte) qui peut nous donner la clé d’une structure fractale.

1.2. Hypothèse d’auto-similarité asymptotique
On formalise l’idée que « chaque nouveau polytope Cn+1C_{n+1}Cn+1​ contient plusieurs copies réduites de CnC_nCn​ ». Par exemple, on peut dire qu’il existe un nombre k≥2k\ge 2k≥2 et un facteur d’échelle ρ∈(0,1)\rho\in (0,1)ρ∈(0,1) tels que, pour un grand nnn,
Cn+1≈⋃i=1k(Φi(Cn)), C_{n+1} \approx \bigcup_{i=1}^k \big( \Phi_i(C_n)\big),Cn+1​≈i=1⋃k​(Φi​(Cn​)),
où Φi\Phi_iΦi​ sont des transformations affines contractantes (i.e. ∥Φi(x)−Φi(y)∥≤ρ∥x−y∥\|\Phi_i(x) - \Phi_i(y)\|\le \rho\|x-y\|∥Φi​(x)−Φi​(y)∥≤ρ∥x−y∥).
Dans la littérature fractale, quand on a une famille d’applications contractantes Φ1,…,Φk\Phi_1,\dots,\Phi_kΦ1​,…,Φk​, il existe un ensemble (dit attracteur fractal) F⊂RmF\subset \mathbb{R}^mF⊂Rm tel que
F=⋃i=1kΦi(F). F = \bigcup_{i=1}^k \Phi_i(F).F=i=1⋃k​Φi​(F).
Si l’on parvient à montrer que {Cn}\{C_n\}{Cn​} converge (dans une topologie appropriée) vers un tel ensemble FFF, et que FFF n’est ni purement de dimension topologique dim⁡top∈N\dim_{\text{top}}\in \mathbb{N}dimtop​∈N ni trop mince (comme un ensemble de mesure zéro trop trivial), alors on peut conclure que FFF est fractal. On se servirait, par exemple, des résultats classiques de théorie des IFS (Iterated Function Systems, Barnsley et Hutchinson, 1981-1982).

2. Critères de fractalité et preuve formelle
Pour qu’un sous-ensemble F⊂RmF \subset \mathbb{R}^mF⊂Rm soit considéré comme « fractal », une définition classique (à la Falconer, ou à la Mandelbrot) repose sur :
Dimension de Hausdorff dim⁡H(F)\dim_{\mathcal{H}}(F)dimH​(F) non entière ou strictement plus grande que sa dimension topologique.
Un certain degré de self-similarité (exacte ou statistique).
La dimension de Hausdorff peut être calculée ou estimée via la méthode de Hutchinson :
Si FFF est l’attracteur d’un système d’applications contractantes {Φi}i=1k\{\Phi_i\}_{i=1}^k{Φi​}i=1k​ avec un facteur d’échelle ρ<1\rho < 1ρ<1 identique (ou ρi\rho_iρi​ variables mais bornées) et certaines conditions de non-recouvrement trop fort (condition OSC — Open Set Condition), alors la dimension de Hausdorff dim⁡H(F)\dim_{\mathcal{H}}(F)dimH​(F) est la solution unique de :
∑i=1kρis  =  1, \sum_{i=1}^k \rho_i^s \;=\; 1,i=1∑k​ρis​=1,
où ρi\rho_iρi​ est le plus grand facteur de contraction de Φi\Phi_iΦi​. Généralement, la solution sss n’est pas un entier, d’où le caractère fractal.

2.1. Argument de la preuve : suite de polytopes CnC_nCn​ convergente
Énoncé : Supposons que la suite (Cn)(C_n)(Cn​) de polytopes (chacun potentiellement en dimension différente, ou projetée dans une dimension ≤m\le m≤m) soit telle que, pour des constantes ρ<1\rho<1ρ<1 et un entier k≥2k\ge2k≥2, on ait :
Cn+1⊂⋃i=1kΦn,i(Cn), C_{n+1} \subset \bigcup_{i=1}^k \Phi_{n,i}(C_n),Cn+1​⊂i=1⋃k​Φn,i​(Cn​),
avec Φn,i\Phi_{n,i}Φn,i​ une application (au moins) contractante, et que le chevauchement entre les Φn,i(Cn)\Phi_{n,i}(C_n)Φn,i​(Cn​) reste contrôlé (afin de satisfaire une version “dynamique” de l’Open Set Condition).
Extraction d’un système contractant :
Si, pour chaque nnn, les Φn,i\Phi_{n,i}Φn,i​ sont εn\varepsilon_nεn​-proches d’applications Φi\Phi_iΦi​ (indépendantes de nnn) et εn→0\varepsilon_n \to 0εn​→0, alors dans la limite n→∞n\to\inftyn→∞, on obtient un système d’applications {Φi}i=1k\{\Phi_i\}_{i=1}^k{Φi​}i=1k​ fixes et contractantes.
Soit FFF l’attracteur fractal de ce système (au sens usuel de la théorie IFS). La suite {Cn}\{C_n\}{Cn​} peut alors être montrée convergente (par exemple, pour la distance de Hausdorff sur les compacts) vers l’ensemble FFF.
Résultat : dim⁡H(F)=s\dim_{\mathcal{H}}(F) = sdimH​(F)=s, où sss est la solution de l’équation de Hutchinson :
∑i=1kρis=1, \sum_{i=1}^k \rho_i^s = 1,i=1∑k​ρis​=1,
(supposé non entier). Alors FFF est fractal, et CnC_nCn​ “devient” fractal dans la limite.
Conséquence : On a donc formellement exhibé un objet (la limite) qui n’a plus de dimension Euclidienne standard, mais une dimension de Hausdorff non entière, possédant un motif de répétition (self-similarité). C’est précisément ce que l’on entend par « fractal » dans un sens rigoureux.

2.2. Hypothèses nécessaires
(H1) Contractions : Les Φn,i\Phi_{n,i}Φn,i​ doivent réellement contracter les distances (ex. affinité avec un facteur ρ<1\rho<1ρ<1).
(H2) Contrôle de recouvrement : Il ne doit pas y avoir un trop grand recouvrement ou une accumulation pathologique (sinon la dimension de Hausdorff peut diverger ou se réduire à un objet trop simple).
(H3) Approximation stable : On suppose que la suite Φn,i\Phi_{n,i}Φn,i​ converge (au moins localement) vers {Φi}i=1k\{\Phi_i\}_{i=1}^k{Φi​}i=1k​, ce qui permet de “geler” la dynamique dans la limite.
Ces hypothèses sont, dans la pratique, difficiles à vérifier précisément pour les véritables cosmohedra ; elles sont plus simples à démontrer pour un modèle qui capture les mêmes règles combinatoires et dont la géométrie (les inégalités) est choisie pour permettre ces propriétés.

3. Application à un « modèle cosmohedron fractal »
Pour aller du formalisme théorique ci-dessus à un exemple concret, on peut définir explicitement :
Un ensemble de variables (t1,…,tn)(t_1, \dots, t_n)(t1​,…,tn​) modélisant les différents canaux d’énergie/moment (analogie aux coupes Feynman).
Un polytope Cn⊂RnC_n\subset \mathbb{R}^nCn​⊂Rn défini par des inégalités du type 0≤t1≤t2≤⋯≤tn≤10 \le t_1 \le t_2 \le \dots \le t_n \le 10≤t1​≤t2​≤⋯≤tn​≤1 et des contraintes supplémentaires (ti+ti+1≤α ti−1+β)(t_i + t_{i+1} \le \alpha\,t_{i-1} + \beta)(ti​+ti+1​≤αti−1​+β), etc.
Règles de subdivision : pour construire Cn+1C_{n+1}Cn+1​ à partir de CnC_nCn​, on rajoute des variables tn+1,…,tn+kt_{n+1}, \dots, t_{n+k}tn+1​,…,tn+k​ et des inégalités analogues, de sorte que la projection sur {t1,…,tn}\{t_1,\dots, t_n\}{t1​,…,tn​} se décompose en “copies” échelonnées de CnC_nCn​.
Exemple schématique :
Cn+1  =  ⋂i=1n+1{(t1,…,tn+1):ti≥0,  ∑j=1n+1tj=1,  … }, C_{n+1} \;=\; \bigcap_{i=1}^{n+1} \bigl\{ (t_1,\dots,t_{n+1}) : t_{i} \ge 0,\; \sum_{j=1}^{n+1} t_j = 1,\; \dots \bigr\},Cn+1​=i=1⋂n+1​{(t1​,…,tn+1​):ti​≥0,j=1∑n+1​tj​=1,…},
avec certaines conditions linéaires (ou affines) introduisant une auto-similarité. On peut prouver que, si on choisit bien les coefficients, alors on obtient une suite {Cn}\{C_n\}{Cn​} satisfaisant les hypothèses (H1), (H2) et (H3).
La preuve de fractalité (au sens Hausdorff) s’articule alors sur la démonstration que la projection (ou section) dans un sous-espace de dimension 2 (ou plus) admet un recouvrement par Φn,i(Cn)\Phi_{n,i}(C_n)Φn,i​(Cn​) avec un rapport d’échelle ρ<1\rho<1ρ<1. Dès lors, la même théorie IFS s’applique et conclut qu’on obtient un attracteur fractal dans la limite.

4. Conclusion et perspectives
Construction rigoureuse :
On a donné le schéma d’une construction formelle (suivant la logique de la théorie IFS) permettant de définir une suite de polytopes s’apparentant à une “version fractale” de cosmohedra. Les conditions de contraction et de non-recouvrement permettent l’utilisation des théorèmes standards de la géométrie fractale (Barnsley, Falconer, Hutchinson).
Preuve de la fractalité :
La démonstration repose sur la convergence vers un attracteur fractal  F\,FF et l’évaluation de la dimension de Hausdorff par la formule ∑i=1kρis=1\sum_{i=1}^k \rho_i^s = 1∑i=1k​ρis​=1. On obtient ainsi, en général, un exposant sss non entier, prouvant le caractère fractal.
Lien avec la physique :
D’un point de vue strictement mathématique, l’existence ou non d’une fractalité dans le vrai “espace de configurations” d’un cosmohedron cosmologique reste conjecturale. Il faudrait prouver qu’en dimension (potentiellement) très élevée et avec des contraintes physiques (pôles d’amplitudes, invariances conformes, etc.), la construction reproduit les conditions de l’IFS.
Remarque finale :
Bien que cette approche donne un cadre théorique pour exhiber un objet fractal (et prouver rigoureusement ses propriétés de fractalité), son application exacte aux cosmohedra décrits dans la littérature de la fonction d’onde cosmologique exigerait des travaux d’adaptation. Néanmoins, c’est ainsi que l’on procéderait pour avoir un argument formel :
Montrer que la “croissance” des polytopes répond à un schéma de self-similarité (au moins asymptotique),
Démontrer, via un théorème standard d’attracteur IFS, que la dimension du lieu-limite est non entière,
Conclure que la structure est fractale.
En résumé, cette construction est rigoureuse si l’on respecte les hypothèses de contraction, de non-recouvrement et de convergence vers des transformations Φi\Phi_iΦi​ fixes. Elle aboutit à une preuve formelle (dans le sens de la théorie des IFS) que le limite de la suite de polytopes est un ensemble fractal, et donc qu’il existe une structure fractale sous-jacente dans ce modèle “cosmohedron-like” à la limite n→∞n \to \inftyn→∞.

"""
resultats_md = main(mon_texte_de_preuve)
